In [0]:
# Databricks Notebook: Silver -> Gold (15m series for chart & 24h change)
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# ========== 설정 ==========
CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
SILVER  = f"{CATALOG}.{SCHEMA}.silver_charts"      # 15m 캔들 원본(Silver)
GOLD_15 = f"{CATALOG}.{SCHEMA}.gold_prices_15m"    # 차트/24h%용 Gold
DAYS_BACK = 10                                      # 최근 N일만 재계산(7일 차트 + 여유)

# ========== 타깃 테이블 생성(없으면) ==========
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {GOLD_15} (
  symbol        STRING,
  interval      STRING,
  open_time     TIMESTAMP,
  close         DOUBLE,
  ma50_15m      DOUBLE,
  ma200_15m     DOUBLE,
  pct_change_24h DOUBLE,
  dt            DATE
) USING DELTA
PARTITIONED BY (dt)
""")

# ========== 소스 로드(증분 범위) ==========
src = (
  spark.table(SILVER)
       .where("open_time IS NOT NULL")
       .where("interval = '15m'")
       .where(f"dt >= date_sub(current_date(), {DAYS_BACK})")
       .select("symbol","interval","open_time","close","dt")
)

# 15m 이동평균 창(행 단위: 50/200개)
w_sym = Window.partitionBy("symbol").orderBy(F.col("open_time"))
w50    = w_sym.rowsBetween(-49, 0)
w200   = w_sym.rowsBetween(-199, 0)

# 24h 변화%: 15분 봉 96개 = 24시간
w96    = w_sym.rowsBetween(-96, -96)

gold_15_df = (
  src
    .withColumn("ma50_15m",   F.avg("close").over(w50))
    .withColumn("ma200_15m",  F.avg("close").over(w200))
    .withColumn("close_24h_ago", F.last("close").over(w96))
    .withColumn(
        "pct_change_24h",
        F.when(F.col("close_24h_ago").isNull(), F.lit(None).cast("double"))
         .otherwise((F.col("close") - F.col("close_24h_ago")) / F.col("close_24h_ago") * 100.0)
    )
    .select("symbol","interval","open_time","close","ma50_15m","ma200_15m","pct_change_24h","dt")
)

# 증분 구간만 MERGE (키: symbol, open_time)
gold_15_df.createOrReplaceTempView("gold15_upserts")
spark.sql(f"""
MERGE INTO {GOLD_15} AS t
USING gold15_upserts AS s
ON  t.symbol = s.symbol AND t.open_time = s.open_time
WHEN MATCHED THEN UPDATE SET
  t.interval       = s.interval,
  t.close          = s.close,
  t.ma50_15m       = s.ma50_15m,
  t.ma200_15m      = s.ma200_15m,
  t.pct_change_24h = s.pct_change_24h,
  t.dt             = s.dt
WHEN NOT MATCHED THEN INSERT *
""")

print(f"[GOLD_15] upsert complete: {GOLD_15}")
